In [2]:
import pandas as pd
import numpy as np
import pyBigWig
from Bio import SeqIO
# 文件路径
gtf_file = "/data/haocheng/data/DNA/Homo_sapiens.GRCh38.104.gtf"
bigwig_file = "/data/haocheng/data/bam/result/GM12878.bigwig"
expression_file = "/data/haocheng/data/gene_expressiom/ENCFF345SHY.tsv"
fasta_file = "/data/haocheng/data/DNA/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
genes = []

# 允许使用不带 'chr' 前缀的染色体名称
valid_chromosomes = [f'chr{i}' for i in range(1, 23)] + ['chrX', 'chrY'] + [str(i) for i in range(1, 23)] + ['X', 'Y']

with open(gtf_file, 'r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        if fields[2] == 'gene':
            info = {x.split(' ')[0]: x.split(' ')[1].strip('";') for x in fields[8].split('; ')}
            if info.get('gene_biotype') == 'protein_coding':
                chrom = fields[0]
                if chrom in valid_chromosomes:
                    tss = int(fields[3])  # TSS是起始位置
                    start = max(0, tss - 10000)  # 向前500个碱基
                    end = tss + 10000  # 向后500个碱基
                    gene_id = info['gene_id']
                    genes.append([chrom, start, end, gene_id])
                #else:
                  # print(f"Skipped non-chromosomal gene: {info['gene_id']} in {chrom}")  # 仅打印非染色体基因的基因ID

genes_df = pd.DataFrame(genes, columns=['chrom', 'start', 'end', 'gene_id'])

# 仅在没有找到基因时打印
if genes_df.empty:
    print("No genes found.")
else:
    print(f"Total genes found: {len(genes_df)}")


Total genes found: 19924


In [3]:
# 读取 TSV 文件
expression_file = "/data/haocheng/data/gene_expressiom/ENCFF345SHY.tsv"
expression_df = pd.read_csv(expression_file, sep='\t')

# 去掉 gene_id 中的版本号后缀
expression_df['gene_id'] = expression_df['gene_id'].str.split('.').str[0]

# 计算 log(TPM + 1)
expression_df['log_TPM'] = np.log1p(expression_df['TPM'])

# 进行合并
merged_df = genes_df.merge(expression_df[['gene_id', 'log_TPM']], on='gene_id', how='inner')

In [4]:
print(merged_df)

      chrom     start       end          gene_id   log_TPM
0         1    675679    695679  ENSG00000284662  0.058269
1         1   1201340   1221340  ENSG00000186827  1.261298
2         1   1193508   1213508  ENSG00000186891  1.214913
3         1   1461765   1481765  ENSG00000160072  2.730464
4         1   6614866   6634866  ENSG00000041988  1.736951
...     ...       ...       ...              ...       ...
19821    21  34503142  34523142  ENSG00000159200  1.795087
19822    21  36146782  36166782  ENSG00000142197  1.749200
19823    21  15719982  15739982  ENSG00000155313  2.461297
19824    21   6489203   6509203  ENSG00000276076  0.048790
19825    21  31108416  31128416  ENSG00000156299  0.482426

[19826 rows x 5 columns]


In [5]:
import pandas as pd

# Load the TSV file
file_path = '/data/haocheng/data/gene_expressiom/ENCFF345SHY.tsv'
df = pd.read_csv(file_path, sep='\t')

# Calculate log(TPM + 1) and find the maximum value
df['log_TPM'] = np.log(df['TPM'] + 1)
max_log_TPM = df['log_TPM'].max()

max_log_TPM

12.396643417132498

In [6]:
import pandas as pd

# Load the TSV file
file_path = '/data/haocheng/data/gene_expressiom/ENCFF345SHY.tsv'
df = pd.read_csv(file_path, sep='\t')

# Calculate log(TPM + 1) and find the maximum value
df['log_TPM'] = np.log(df['TPM'] + 1)
max_log_TPM = df['log_TPM'].max()

max_log_TPM
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the TSV file
file_path = '/data/haocheng/data/gene_expressiom/ENCFF345SHY.tsv'
df = pd.read_csv(file_path, sep='\t')

# Calculate log(TPM + 1)
df['log_TPM'] = np.log(df['TPM'] + 1)

# Plot the distribution of log_TPM
plt.figure(figsize=(10, 6))
plt.hist(df['log_TPM'], bins=200, color='blue', alpha=0.7)
plt.title('Distribution of log(TPM + 1)')
plt.xlabel('log(TPM + 1)')
plt.ylabel('Frequency')
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [7]:
import pandas as pd
import numpy as np
import pyBigWig
from Bio import SeqIO
import torch
# 读取BigWig文件
bw = pyBigWig.open(bigwig_file)

# 创建一个字典用于快速查找FASTA序列
fasta_sequences = {record.id: str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")}

# 定义碱基到 one-hot 编码的映射
mapping = {
    'A': [1, 0, 0, 0, 0],
    'C': [0, 1, 0, 0, 0],
    'G': [0, 0, 1, 0, 0],
    'T': [0, 0, 0, 1, 0],
    'N': [0, 0, 0, 0, 1]
}

# 创建一个用于存储结果的列表
results = []

# 遍历merged_df的每一行
for index, row in merged_df.iterrows():
    chrom = row["chrom"]  # 不再添加 "chr" 前缀
    start = row["start"]
    end = row["end"]

    # 提取FASTA序列
    sequence = fasta_sequences[chrom][start:end]  # 提取对应区间的序列

    # 转换为 one-hot 编码
    one_hot = np.array([mapping.get(nuc, [0, 0, 0, 0, 0]) for nuc in sequence], dtype=np.float32)

    # 获取BigWig值
    bigwig_values = bw.values(f'chr{chrom}', start, end)

    # 添加BigWig值作为最后一行
    combined_matrix = np.vstack([one_hot.T, bigwig_values])

    # 添加到结果列表
    results.append(combined_matrix)

# 关闭BigWig文件
bw.close()

# 转换为PyTorch张量
results_tensor = torch.tensor(results, dtype=torch.float32)

/tmp/ipykernel_1109585/715821556.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  results_tensor = torch.tensor(results, dtype=torch.float32)


In [8]:
# 提取 BigWig 值（第六行）
bigwig_values = results_tensor[:, 5, :]  # 选择第六行

# 找到最大值和其索引
max_value, max_index = torch.max(bigwig_values, dim=1)

# 打印最大值及其索引
print("Max BigWig values:", max_value)
print("Indices of max BigWig values:", max_index)
print(bigwig_values)

Max BigWig values: tensor([ 190., 1349., 1349.,  ...,  389.,  989.,  419.])
Indices of max BigWig values: tensor([ 441,    0, 7468,  ..., 9699, 9667,  415])
tensor([[   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [1349., 1349., 1349.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.]])


In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dense, Add, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


2024-08-12 19:23:40.365841: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 19:23:40.774770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 19:23:42.622527: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 19:23:46.452677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
# 划分训练集和测试集
test_chromosomes = ['13', '14', '15']  # 选择要作为测试集的染色体
test_mask = merged_df['chrom'].isin(test_chromosomes)  # 测试集掩码
train_mask = ~test_mask  # 训练集掩码

# 创建训练集和测试集的 DataFrame
train_df = merged_df[train_mask]
test_df = merged_df[test_mask]

# 确保特征矩阵与 DataFrame 行对应
X_train = results_tensor[train_mask].numpy()  # 获取训练集的输入特征
y_train = train_df['log_TPM'].values
X_test = results_tensor[test_mask].numpy()  # 获取测试集的输入特征
y_test = test_df['log_TPM'].values

# 直接在最后一个维度添加一个维度
X_train = np.expand_dims(X_train, axis=-1)  # 将形状变为 (1525, 6, 20000, 1)
X_test = np.expand_dims(X_test, axis=-1)    # 将形状变为 (18301, 6, 20000, 1)

# 划分训练集和验证集
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [12]:
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ReduceLROnPlateau
# 学习率调整
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# 标准化数据
scaler = StandardScaler()

# 拟合并转换训练集
X_train_final = scaler.fit_transform(X_train_final.reshape(-1, X_train_final.shape[-1])).reshape(X_train_final.shape)

# 转换验证集
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# 构建 CNN 模型
input_layer = Input(shape=(X_train_final.shape[1], X_train_final.shape[2]))

# 第一层卷积
conv1 = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
conv1 = Dropout(0.2)(conv1)

# 第二层卷积
conv2 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv1)
conv2 = Dropout(0.2)(conv2)

# 平铺并添加全连接层
flatten = Flatten()(conv2)  # 注意这里使用的是conv2
dense = Dense(50, activation='relu')(flatten)
output_layer = Dense(1)(dense)

# 构建模型
model = Model(inputs=input_layer, outputs=output_layer)

# 设置优化器
optimizer = Adam(learning_rate=0.0001)  # 更低的学习率
model.compile(optimizer=optimizer, loss='mae')  # 使用均方误差作为损失函数

# 自定义回调类，用于在每个周期结束时打印评估指标
class CustomMetrics(Callback):
    def __init__(self, X_test, y_test, threshold=0.1):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test)
        mse = mean_squared_error(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)
        r2 = r2_score(self.y_test, y_pred)
        pearson_corr, _ = pearsonr(self.y_test, y_pred.flatten())
        
        accuracy = np.mean(np.abs(y_pred.flatten() - self.y_test) < self.threshold)
        
        print(f"Epoch {epoch + 1}: MSE = {mse:.4f}, MAE = {mae:.4f}, R² = {r2:.4f}, Pearson Correlation = {pearson_corr:.4f}, Accuracy = {accuracy:.4f}")

# 训练模型，使用自定义回调
custom_metrics = CustomMetrics(X_test, y_test)
model.fit(X_train_final, y_train_final, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[custom_metrics, lr_reduction])

Epoch 1/100
48/48 [==============================] - 1s 15ms/step
Epoch 1: MSE = 2123.3692, MAE = 28.0010, R² = -975.0362, Pearson Correlation = 0.2380, Accuracy = 0.0092
458/458 [==============================] - 22s 44ms/step - loss: 1.1807 - val_loss: 1.1448 - lr: 1.0000e-04
Epoch 2/100
48/48 [==============================] - 1s 19ms/step
Epoch 2: MSE = 2041.7852, MAE = 28.8085, R² = -937.5350, Pearson Correlation = 0.2955, Accuracy = 0.0059
458/458 [==============================] - 19s 42ms/step - loss: 1.1113 - val_loss: 1.0704 - lr: 1.0000e-04
Epoch 3/100
48/48 [==============================] - 1s 17ms/step
Epoch 3: MSE = 1220.2966, MAE = 20.5566, R² = -559.9263, Pearson Correlation = 0.1653, Accuracy = 0.0026
458/458 [==============================] - 19s 42ms/step - loss: 1.0760 - val_loss: 1.0392 - lr: 1.0000e-04
Epoch 4/100
48/48 [==============================] - 1s 17ms/step
Epoch 4: MSE = 518.2017, MAE = 13.9532, R² = -237.1986, Pearson Correlation = 0.1389, Accuracy = 

In [15]:
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Conv1D, Dropout, Flatten, Dense, Add
from keras.models import Model
from keras import backend as K

# 学习率调整
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# 标准化数据
scaler = StandardScaler()

# 拟合并转换训练集
X_train_final = scaler.fit_transform(X_train_final.reshape(-1, X_train_final.shape[-1])).reshape(X_train_final.shape)

# 转换验证集
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# 残差块
def residual_block(x, filters, kernel_size):
    shortcut = x  # 保存输入以便后续添加
    x = Conv1D(filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(filters, kernel_size=kernel_size, activation=None, padding='same')(x)  # 不激活

    # 如果输入和输出形状不同，使用1x1卷积调整形状
    if K.int_shape(shortcut)[-1] != K.int_shape(x)[-1]:
        shortcut = Conv1D(filters, kernel_size=1, padding='same')(shortcut)

    x = Add()([x, shortcut])  # 添加残差连接
    x = K.relu(x)  # 使用ReLU激活
    return x

# 构建 CNN 模型
input_layer = Input(shape=(X_train_final.shape[1], X_train_final.shape[2]))

# 第一层卷积
conv1 = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
conv1 = Dropout(0.2)(conv1)

# 残差块，滤波器数量依次增加
res_block1 = residual_block(conv1, 64, kernel_size=3)
res_block2 = residual_block(res_block1, 128, kernel_size=3)
res_block3 = residual_block(res_block2, 256, kernel_size=3)
res_block4 = residual_block(res_block3, 512, kernel_size=3)  # 增加到512

# 平铺并添加全连接层
flatten = Flatten()(res_block4)
dense = Dense(50, activation='relu')(flatten)
output_layer = Dense(1)(dense)  # 输出层

# 构建模型
model = Model(inputs=input_layer, outputs=output_layer)

# 设置优化器
optimizer = Adam(learning_rate=0.0001)  # 更低的学习率
model.compile(optimizer=optimizer, loss='mae')  # 使用均方误差作为损失函数

# 自定义回调类，用于在每个周期结束时打印评估指标
class CustomMetrics(Callback):
    def __init__(self, X_test, y_test, threshold=0.1):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test)
        mse = mean_squared_error(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)
        r2 = r2_score(self.y_test, y_pred)
        pearson_corr, _ = pearsonr(self.y_test, y_pred.flatten())
        
        accuracy = np.mean(np.abs(y_pred.flatten() - self.y_test) < self.threshold)
        
        print(f"Epoch {epoch + 1}: MSE = {mse:.4f}, MAE = {mae:.4f}, R² = {r2:.4f}, Pearson Correlation = {pearson_corr:.4f}, Accuracy = {accuracy:.4f}")

# 训练模型，使用自定义回调
custom_metrics = CustomMetrics(X_test, y_test)
model.fit(X_train_final, y_train_final, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[custom_metrics, lr_reduction])


Epoch 1/100
572/572 [==============================] - 6s 10ms/step
Epoch 1: MSE = 1135.0163, MAE = 21.4017, R² = -490.9135, Pearson Correlation = 0.3968, Accuracy = 0.0079
458/458 [==============================] - 32s 64ms/step - loss: 1.0895 - val_loss: 1.0144 - lr: 1.0000e-04
Epoch 2/100
572/572 [==============================] - 6s 10ms/step
Epoch 2: MSE = 203.2046, MAE = 7.7527, R² = -87.0684, Pearson Correlation = 0.3377, Accuracy = 0.0257
458/458 [==============================] - 26s 58ms/step - loss: 1.0445 - val_loss: 1.0235 - lr: 1.0000e-04
Epoch 3/100
572/572 [==============================] - 6s 10ms/step
Epoch 3: MSE = 249.2598, MAE = 8.9626, R² = -107.0286, Pearson Correlation = 0.3604, Accuracy = 0.0123
458/458 [==============================] - 27s 59ms/step - loss: 1.0310 - val_loss: 1.0020 - lr: 1.0000e-04
Epoch 4/100
572/572 [==============================] - 6s 10ms/step
Epoch 4: MSE = 160.6360, MAE = 7.1297, R² = -68.6193, Pearson Correlation = 0.3601, Accuracy =

: 

In [13]:
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, LSTM, Dropout, Dense
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np

# 学习率调整
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# 标准化数据
scaler = StandardScaler()

# 拟合并转换训练集
X_train_final = scaler.fit_transform(X_train_final.reshape(-1, X_train_final.shape[-1])).reshape(X_train_final.shape)

# 转换验证集
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# 构建 LSTM 模型
input_layer = Input(shape=(X_train_final.shape[1], X_train_final.shape[2]))

# LSTM 层
lstm_layer = LSTM(128, return_sequences=True)(input_layer)  # 这里可以调整单元数
lstm_layer = Dropout(0.2)(lstm_layer)
lstm_layer = LSTM(64)(lstm_layer)  # 最后一层LSTM，输出为64维

# 添加全连接层
dense = Dense(50, activation='relu')(lstm_layer)
output_layer = Dense(1)(dense)  # 输出层

# 构建模型
model = Model(inputs=input_layer, outputs=output_layer)

# 设置优化器
optimizer = Adam(learning_rate=0.0001)  # 更低的学习率
model.compile(optimizer=optimizer, loss='mae')  # 使用均方误差作为损失函数

# 自定义回调类，用于在每个周期结束时打印评估指标
class CustomMetrics(Callback):
    def __init__(self, X_test, y_test, threshold=0.1):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test)
        mse = mean_squared_error(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)
        r2 = r2_score(self.y_test, y_pred)
        pearson_corr, _ = pearsonr(self.y_test, y_pred.flatten())
        
        accuracy = np.mean(np.abs(y_pred.flatten() - self.y_test) < self.threshold)
        
        print(f"Epoch {epoch + 1}: MSE = {mse:.4f}, MAE = {mae:.4f}, R² = {r2:.4f}, Pearson Correlation = {pearson_corr:.4f}, Accuracy = {accuracy:.4f}")

# 训练模型，使用自定义回调
custom_metrics = CustomMetrics(X_test, y_test)
model.fit(X_train_final, y_train_final, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[custom_metrics, lr_reduction])


Epoch 1/100
48/48 [==============================] - 2s 33ms/step
Epoch 1: MSE = 4.7198, MAE = 1.6146, R² = -1.1695, Pearson Correlation = 0.1715, Accuracy = 0.0938
458/458 [==============================] - 56s 115ms/step - loss: 1.0473 - val_loss: 1.0270 - lr: 1.0000e-04
Epoch 2/100
48/48 [==============================] - 1s 29ms/step
Epoch 2: MSE = 4.2969, MAE = 1.5016, R² = -0.9751, Pearson Correlation = 0.1408, Accuracy = 0.2413
458/458 [==============================] - 51s 112ms/step - loss: 1.0081 - val_loss: 1.0007 - lr: 1.0000e-04
Epoch 3/100
48/48 [==============================] - 2s 31ms/step
Epoch 3: MSE = 4.1642, MAE = 1.4803, R² = -0.9141, Pearson Correlation = 0.1185, Accuracy = 0.1659
458/458 [==============================] - 49s 107ms/step - loss: 0.9895 - val_loss: 1.0081 - lr: 1.0000e-04
Epoch 4/100
48/48 [==============================] - 2s 34ms/step
Epoch 4: MSE = 4.6204, MAE = 1.5719, R² = -1.1238, Pearson Correlation = 0.0275, Accuracy = 0.2170
458/458 [====

KeyboardInterrupt: 